# Usage of `src.data.iem_query` module.

This notebook outlies the basic usage of the `src.data.iem_query` module. Used to query US weather stations data from the IEM database.
 
**Requirements**
 - A `.yml` file being a copy of `config/config_template.yml` contaning information about the flights to be downloaded, as well as a path to the credentials config file

**Helpful Links**

 - [Downloading and Interpreting IEM Data Wiki](https://github.com/Flight-Path-Analysis/FlightPathAnalysis/wiki/Downloading_and_Interpreting_IEM_Data)

 - [Iowa Environmental Mesonet](https://mesonet.agron.iastate.edu/)

## Basic Setup

In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for downloading IEM data 
from src.data import iem_query
# Utilities package
from src.common import utils

# Normalizing all paths to work on all operational systems
root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file as a dictionary
with open(config_path, 'r', encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=clear)

iem_querier = iem_query.Querier(config, logger=logger)


## Loading all stations and station properties

This code loads information for all weather stations in the US states. It returns the results in the form of a Pandas DataFrame.

It also, by default, cleans up the data to a more interpretable format

The columns of the dataframe are described below

  - **id**: [Identification Number] - The unique identifier assigned to each weather station.
  - **elevation**: [Elevation] - The height of the weather station above sea level, usually measured in meters or feet.
  - **sname**: [Station Name] - The official name of the weather station.
  - **time_domain**: [Time Domain] - The time zone or specific time reference in which the weather station operates or reports data.
  - **archive_begin**: [Archive Start Date] - The date when the weather station began archiving data.
  - **archive_end**: [Archive End Date] - The last date for which data is available in the archive (if the station is still operational, this might be ongoing).
  - **state**: [State] - The state or province where the weather station is located.
  - **country**: [Country] - The country where the weather station is situated.
  - **climate_site**: [Climate Site Identifier] - A unique code or identifier for the climate data site if it is part of a broader climate observation network.
  - **wfo**: [Weather Forecast Office] - The local National Weather Service office responsible for the area where the weather station is located.
  - **tzname**: [Time Zone Name] - The name of the time zone where the weather station is located, such as 'EST' for Eastern Standard Time.
  - **ncdc81**: [NCDC 1981-2010 Normals] - Reference to the National Climatic Data Center's climate normals for the period from 1981 to 2010.
  - **ncei91**: [NCEI 1991-2020 Normals] - Reference to the National Centers for Environmental Information's climate normals for the period from 1991 to 2020.
  - **ugc_county**: [UGC County Code] - The Universal Geographic Code for the county in which the weather station is located.
  - **ugc_zone**: [UGC Zone Code] - The Universal Geographic Code for the forecast zone in which the weather station is located.
  - **county**: [County] - The county in which the weather station is located.
  - **network**: [Network] - The network or system of weather stations to which this particular station belongs.
  - **lon**: [Longitude] - The longitudinal coordinate of the weather station's location, usually in decimal degrees.
  - **lat**: [Latitude] - The latitudinal coordinate of the weather station's location, usually in decimal degrees.
  - **sigma**: [Sigma] - A custom measure, indicating the width (in physical distance) of the influence of this weather station for interpolation purposes.



In [ ]:
# Loads the information for all US weather stations.
# filters to keep only stations for which there's available data
# Dates are specified in config['base-configs']
stations = iem_querier.load_all_station_properties(filter_by_date=True)

stations

In [ ]:
stations.columns

## Loading data for a particular station in a date range

This code loads weather data for a specific weather station. It returns the results in the form of a Pandas DataFrame.

It also, by default, cleans up the data to a more interpretable format.

The columns of the dataframe are described below

  - **station:** The identifier for the weather station, usually an alphanumeric code.
  
  - **valid:** The timestamp indicating when the observation was taken.

  - **lon:** Longitude of the weather station.

  - **lat:** Latitude of the weather station.

  - **tmpf:** Temperature in degrees Fahrenheit.

  - **dwpf:** Dew point temperature in degrees Fahrenheit.

  - **relh:** Relative humidity, typically given as a percentage.

  - **drct:** Wind direction in degrees, with 0° or 360° representing north.

  - **sknt:** Wind speed in knots.

  - **p01i:** Precipitation for the past 1 hour, usually in inches.

  - **alti:** Altimeter setting, which is the current pressure at sea level.

  - **mslp:** Mean Sea Level Pressure.

  - **vsby:** Visibility, typically in statute miles.

  - **gust:** Wind gust speed in knots.

  - **skyc1, skyc2, skyc3, skyc4:** Sky cover for up to four levels. This can be in terms of clear (CLR), scattered (SCT), broken (BKN), overcast (OVC), etc.

  - **skyl1, skyl2, skyl3, skyl4:** Altitude (usually in feet) at which the respective sky cover (skyc1-4) levels are observed.

  - **wxcodes:** Weather codes representing specific weather phenomena, like rain (RA), snow (SN), etc.

  - **ice_accretion_1hr:** Amount of ice accretion (ice buildup) in the past 1 hour.

  - **ice_accretion_3hr:** Amount of ice accretion in the past 3 hours.

  - **ice_accretion_6hr:** Amount of ice accretion in the past 6 hours.

  - **peak_wind_gust:** Peak wind gust observed during the period.

  - **peak_wind_drct:** Direction from which the peak wind gust came.

  - **peak_wind_time:** Time when the peak wind gust was observed.

  - **feel:** "Feels like" temperature, which takes into account factors like wind chill or heat index.

  - **metar:** METAR is a format for reporting weather information, mainly used by pilots. This column would contain the raw METAR string for the observation.

  - **snowdepth:** Depth of snow on the ground, typically measured in inches.

In [ ]:
# Loads weather data for the specific weather station fed as an argument
station_data = iem_querier.query_station_data(stations.iloc[0])
station_data

In [ ]:
# Loads weather data for all weather_stations | It will take a while
all_station_data = iem_querier.query_multiple_station_data(stations)
all_station_data